In [ ]:
# filtering google trurh values

import pandas as pd

df_fmain = pd.read_csv('raw_datasets/supplementary_dataset_TS/epidemiology.csv')

# Define the regex pattern for US states
pattern = r'^US_[A-Z]{2}$'

# Create a mask, replacing NaN values with False
mask = df_fmain['location_key'].str.match(pattern).fillna(False)

# Apply the mask to filter rows
df_filtered = df_fmain[mask]

# Continue with your intended groupby and aggregation operation
# (Ensure 'date' and 'new_confirmed' columns exist in your DataFrame)
grouped_df_fmain = df_filtered.groupby(['date', 'location_key']).agg({
    'new_confirmed': 'first',  # Adjust the aggregation as necessary
}).reset_index()

# Save the grouped DataFrame
grouped_df_fmain.to_csv('processed_data/unique_truth_google_dataset.csv', index=False)


In [ ]:
# time seris correlation ranking main dataset

import os
import pandas as pd
import statsmodels.api as sm

# Adjust these paths to where your datasets are located
truth_data_path = 'processed_data/unique_truth_main_dataset.csv'
datasets_folder_path = 'raw_datasets/supplementary_dataset_TS'

# Load the truth data
truth_data = pd.read_csv(truth_data_path)
truth_data.rename(columns={'abbreviation': 'location_key', 'target_end_date': 'date'}, inplace=True)
truth_data['location_key'] = 'US_' + truth_data['location_key']

# Store results
results = []

# Iterate over each dataset in the folder
for filename in os.listdir(datasets_folder_path):
    if filename.endswith('.csv'):
        dataset_path = os.path.join(datasets_folder_path, filename)
        data = pd.read_csv(dataset_path)
        
        print(f"Processing {dataset_path}...")
        
        # Merge with truth_data on date and location
        merged_data = pd.merge(truth_data, data, on=['date', 'location_key'])
        
        for column in data.columns:
            if column not in ['date', 'location_key']:
                # Drop rows where the current column or truth_value has NaN
                clean_merged_data = merged_data.dropna(subset=[column, 'truth_value'])

                # Check if there's still data left after dropping NaNs
                if not clean_merged_data.empty:
                    # Check if the data is numeric
                    if pd.api.types.is_numeric_dtype(clean_merged_data[column]):
                        # Prepare data for regression
                        X = sm.add_constant(clean_merged_data[column])  # Add constant
                        y = clean_merged_data['truth_value']

                        # Run linear regression
                        model = sm.OLS(y, X).fit()

                        # Record the result
                        results.append({
                            'filename': filename,
                            'variable': column,
                            'correlation': model.rsquared
                        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Sort results by correlation in descending order
results_df = results_df.sort_values(by='correlation', ascending=False)

results_df.to_csv('processed_data/OLS_correlation_ranking.csv', index=False)

# Output the sorted results
results_df.head()

In [ ]:
# ranked data merging main dataset

import os
import pandas as pd

# Load the truth data
truth_data_path = 'processed_data/unique_truth_main_dataset.csv'
datasets_folder_path = 'raw_datasets/supplementary_dataset_TS'
truth_data = pd.read_csv(truth_data_path)
truth_data.rename(columns={'abbreviation': 'location_key', 'target_end_date': 'date'}, inplace=True)
truth_data['location_key'] = 'US_' + truth_data['location_key']

# Load the correlation data
correlation_data_path = 'processed_data\OLS_correlation_ranking.csv'
correlation_data = pd.read_csv(correlation_data_path)

# Filter variables with R^2 greater than 0.04
selected_variables = correlation_data[correlation_data['correlation'] > 0.04]

# Iterate over each dataset in the folder
for filename in os.listdir(datasets_folder_path):
    if filename.endswith('.csv'):
        dataset_path = os.path.join(datasets_folder_path, filename)
        data = pd.read_csv(dataset_path)
        
        print(f"Processing {dataset_path}...")
        
        # Process only selected variables for the current dataset
        for _, row in selected_variables.iterrows():
            if row['filename'] == filename:
                column = row['variable']
                
                # Verify if the column exists in the current dataset
                if column in data.columns:
                    # Create a temporary DataFrame to hold the current column's data
                    temp_data = data[['date', 'location_key', column]].dropna()

                    # Merge the temporary DataFrame with the truth_data DataFrame
                    truth_data = pd.merge(truth_data, temp_data, on=['date', 'location_key'], how='left', suffixes=('', '_temp'))

                    # Check if the merged column exists (avoid overwriting existing data)
                    if f'{column}_temp' in truth_data.columns:
                        # If the column already exists in truth_data, we combine the original and new columns
                        truth_data[column] = truth_data[column].combine_first(truth_data[f'{column}_temp'])
                        # Drop the temporary column after combining
                        truth_data.drop(columns=[f'{column}_temp'], inplace=True)

# After processing all files, save the updated truth_data DataFrame
truth_data.to_csv('processed_data/ranked_merged_dataset_main.csv', index=False)

truth_data.head()

In [ ]:
import pandas as pd
import numpy as np

results_df = pd.read_csv('processed_data/ranked_merged_dataset_main.csv')

missing_value_percentages = results_df.select_dtypes(include=[np.number]).isnull().mean().sort_values() * 100

print(missing_value_percentages)

In [ ]:
# Main dataset

import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
final_dataset = pd.read_csv('processed_data/ranked_merged_dataset.csv')
final_dataset.drop(columns=['location_name'], inplace=True)

# Load correlation data to filter variables
selected_variables = [
    'cumulative_deceased',
    'new_confirmed',
    'cumulative_confirmed',
    'new_hospitalized_patients',
    'current_hospitalized_patients',
    'cumulative_hospitalized_patients',
    'current_intensive_care_patients',
    'cumulative_vaccine_doses_administered',
    'cumulative_persons_vaccinated',
    'cumulative_persons_fully_vaccinated',
    'new_tested',
    'cumulative_tested',
    'cumulative_recovered',
    'new_recovered',
    'location_key_US_AK', 'location_key_US_AL', 'location_key_US_AR', 'location_key_US_AZ', 'location_key_US_CA', 'location_key_US_CO', 'location_key_US_CT', 'location_key_US_DE', 'location_key_US_FL', 'location_key_US_GA', 'location_key_US_HI', 'location_key_US_IA', 'location_key_US_ID', 'location_key_US_IL', 'location_key_US_IN', 'location_key_US_KS', 'location_key_US_KY', 'location_key_US_LA', 'location_key_US_MA', 'location_key_US_MD', 'location_key_US_ME', 'location_key_US_MI', 'location_key_US_MN', 'location_key_US_MO', 'location_key_US_MS', 'location_key_US_MT', 'location_key_US_NC', 'location_key_US_ND', 'location_key_US_NE', 'location_key_US_NH', 'location_key_US_NJ', 'location_key_US_NM', 'location_key_US_NV', 'location_key_US_NY', 'location_key_US_OH', 'location_key_US_OK', 'location_key_US_OR', 'location_key_US_PA', 'location_key_US_PR', 'location_key_US_RI', 'location_key_US_SC', 'location_key_US_SD', 'location_key_US_TN', 'location_key_US_TX', 'location_key_US_UT', 'location_key_US_VA', 'location_key_US_VT', 'location_key_US_WA', 'location_key_US_WI', 'location_key_US_WV', 'location_key_US_WY'
]

pre_selected_variables = [
    'cumulative_deceased',
    'new_confirmed',
    'cumulative_confirmed',
    'new_hospitalized_patients',
    'current_hospitalized_patients',
    'cumulative_hospitalized_patients',
    'current_intensive_care_patients',
    'cumulative_vaccine_doses_administered',
    'cumulative_persons_vaccinated',
    'cumulative_persons_fully_vaccinated',
    'new_tested',
    'cumulative_tested',
    'cumulative_recovered',
    'new_recovered'
]

# Impute missing values for numerical columns
for column in pre_selected_variables:
    if final_dataset[column].dtype != 'object':  # If column is numerical
        final_dataset[column].fillna(final_dataset[column].mean(), inplace=True)
        
X = X.apply(pd.to_numeric, errors='coerce')

X.fillna(X.mean(), inplace=True)

y = pd.to_numeric(y, errors='coerce').fillna(y.mean())

# Create dummy variables for 'location_key'
final_dataset = pd.get_dummies(final_dataset, columns=['location_key'], dtype=int)

# Define independent variables (X) and dependent variable (y)
# Ensure 'location_key' dummies are included in X
X = final_dataset[[col for col in final_dataset.columns if col in selected_variables or 'location_key_' in col]]
y = final_dataset['truth_value']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Use sm.OLS to perform the regression and fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())


In [ ]:
import os
import pandas as pd
import statsmodels.api as sm

# Adjust these paths to where your datasets are located
truth_data_path = 'processed_data/unique_truth_google_dataset.csv'
datasets_folder_path = 'raw_datasets/supplementary_dataset_TS'

# Load the truth data
truth_data = pd.read_csv(truth_data_path)

# Ensure 'new_confirmed' is in truth_data
if 'new_confirmed' not in truth_data.columns:
    raise ValueError("Column 'new_confirmed' not found in truth data.")

results = []

# Iterate over each dataset in the folder
for filename in os.listdir(datasets_folder_path):
    if filename.endswith('.csv'):
        dataset_path = os.path.join(datasets_folder_path, filename)
        data = pd.read_csv(dataset_path)

        # Debug: Print columns to verify structure
        print(f"Processing {filename}, columns: {data.columns.tolist()}")

        # Merge with truth_data on date and location
        merged_data = pd.merge(truth_data, data, on=['date', 'location_key'])

        # Ensure 'new_confirmed' is in merged_data
        if 'new_confirmed' not in merged_data.columns:
            print(f"Warning: 'new_confirmed' not found after merging with {filename}. Skipping this file.")
            continue  # Skip this file if 'new_confirmed' is not in the merged data

        for column in data.columns:
            if column not in ['date', 'location_key']:
                try:
                    # Drop rows where the current column or truth_value has NaN
                    clean_merged_data = merged_data.dropna(subset=[column, 'new_confirmed'])

                    if not clean_merged_data.empty:
                        if pd.api.types.is_numeric_dtype(clean_merged_data[column]):
                            X = sm.add_constant(clean_merged_data[column])  # Add constant
                            y = clean_merged_data['new_confirmed']

                            model = sm.OLS(y, X).fit()

                            results.append({
                                'filename': filename,
                                'variable': column,
                                'correlation': model.rsquared
                            })
                except KeyError as e:
                    print(f"KeyError encountered for column {e} in file {filename}. It might be missing after the merge.")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='correlation', ascending=False)
results_df.to_csv('processed_data/Google_OLS_correlation_ranking.csv', index=False)
print(results_df.head())

In [ ]:
# ranked data merging main dataset

import os
import pandas as pd

# Load the truth data
truth_data_path = 'processed_data/unique_truth_google_dataset.csv'
datasets_folder_path = 'raw_datasets/supplementary_dataset_TS'
truth_data = pd.read_csv(truth_data_path)

# Load the correlation data
correlation_data_path = 'processed_data/Google_OLS_correlation_ranking.csv'
correlation_data = pd.read_csv(correlation_data_path)

# Filter variables with R^2 greater than 0.04
selected_variables = correlation_data[correlation_data['correlation'] > 0.2]

# Iterate over each dataset in the folder
for filename in os.listdir(datasets_folder_path):
    if filename.endswith('.csv'):
        dataset_path = os.path.join(datasets_folder_path, filename)
        data = pd.read_csv(dataset_path)
        
        print(f"Processing {dataset_path}...")
        
        # Process only selected variables for the current dataset
        for _, row in selected_variables.iterrows():
            if row['filename'] == filename:
                column = row['variable']
                
                # Verify if the column exists in the current dataset
                if column in data.columns:
                    # Create a temporary DataFrame to hold the current column's data
                    temp_data = data[['date', 'location_key', column]].dropna()

                    # Merge the temporary DataFrame with the truth_data DataFrame
                    truth_data = pd.merge(truth_data, temp_data, on=['date', 'location_key'], how='left', suffixes=('', '_temp'))

                    # Check if the merged column exists (avoid overwriting existing data)
                    if f'{column}_temp' in truth_data.columns:
                        # If the column already exists in truth_data, we combine the original and new columns
                        truth_data[column] = truth_data[column].combine_first(truth_data[f'{column}_temp'])
                        # Drop the temporary column after combining
                        truth_data.drop(columns=[f'{column}_temp'], inplace=True)

# After processing all files, save the updated truth_data DataFrame
truth_data.to_csv('processed_data/Google_ranked_merged_dataset.csv', index=False)

truth_data.head()

In [3]:
import pandas as pd
import numpy as np

results_df = pd.read_csv('processed_data/Google_ranked_merged_dataset.csv')

missing_value_percentages = results_df.select_dtypes(include=[np.number]).isnull().mean().sort_values() * 100

print(missing_value_percentages)

new_confirmed                                           0.000000
new_hospitalized_patients                               6.508908
current_hospitalized_patients                          12.399941
current_intensive_care_patients                        16.455790
current_ventilator_patients                            83.167952
new_intensive_care_patients                            92.434247
cumulative_intensive_care_patients                     93.011546
new_confirmed_age_2                                    94.332141
new_confirmed_age_0                                    94.332141
new_confirmed_age_3                                    94.332141
new_confirmed_age_1                                    94.332141
new_deceased_age_9                                     94.525803
lawatlas_home_except_obtaining_necessary_supplies      96.021616
lawatlas_home_except_engaging_in_outdoor_activities    96.021616
new_confirmed_age_6                                    96.775978
new_confirmed_age_7      

In [7]:
# Google dataset

import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
final_dataset = pd.read_csv('processed_data/Google_ranked_merged_dataset.csv')

# Load correlation data to filter variables
selected_variables = [
    'new_hospitalized_patients',
    'current_hospitalized_patients',
    'current_intensive_care_patients',
    'location_key_US_AK', 'location_key_US_AL', 'location_key_US_AR', 'location_key_US_AZ', 'location_key_US_CA', 'location_key_US_CO', 'location_key_US_CT', 'location_key_US_DE', 'location_key_US_FL', 'location_key_US_GA', 'location_key_US_HI', 'location_key_US_IA', 'location_key_US_ID', 'location_key_US_IL', 'location_key_US_IN', 'location_key_US_KS', 'location_key_US_KY', 'location_key_US_LA', 'location_key_US_MA', 'location_key_US_MD', 'location_key_US_ME', 'location_key_US_MI', 'location_key_US_MN', 'location_key_US_MO', 'location_key_US_MS', 'location_key_US_MT', 'location_key_US_NC', 'location_key_US_ND', 'location_key_US_NE', 'location_key_US_NH', 'location_key_US_NJ', 'location_key_US_NM', 'location_key_US_NV', 'location_key_US_NY', 'location_key_US_OH', 'location_key_US_OK', 'location_key_US_OR', 'location_key_US_PA', 'location_key_US_PR', 'location_key_US_RI', 'location_key_US_SC', 'location_key_US_SD', 'location_key_US_TN', 'location_key_US_TX', 'location_key_US_UT', 'location_key_US_VA', 'location_key_US_VT', 'location_key_US_WA', 'location_key_US_WI', 'location_key_US_WV', 'location_key_US_WY'
]

pre_selected_variables = [
    'new_hospitalized_patients',
    'current_hospitalized_patients',
    'current_intensive_care_patients'
]

for column in pre_selected_variables:
    if final_dataset[column].dtype != 'object':  # If column is numerical
        final_dataset[column].fillna(final_dataset[column].mean(), inplace=True)

# Assuming 'location_key' needs to be converted into dummies and included in the regression
final_dataset = pd.get_dummies(final_dataset, columns=['location_key'], dtype=int)

# Now define X and y
selected_variables = pre_selected_variables + [col for col in final_dataset.columns if 'location_key_' in col]

X = final_dataset[selected_variables]  # Ensure this includes all your independent variables
y = final_dataset['new_confirmed'].apply(pd.to_numeric, errors='coerce')  # Adjust column name as needed

# Add constant to X
X = sm.add_constant(X)

# Handle any potential NaN values in X or y before regression
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

# Perform the regression
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          new_confirmed   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     847.2
Date:                Tue, 05 Mar 2024   Prob (F-statistic):               0.00
Time:                        12:22:47   Log-Likelihood:            -5.2593e+05
No. Observations:               54218   AIC:                         1.052e+06
Df Residuals:                   54159   BIC:                         1.053e+06
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     